In [1]:
import pandas as pd
import chardet           # para detectar la codificación de texto en archivos
import numpy as np

from sklearn import preprocessing, model_selection, metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras import optimizers

from matplotlib import pyplot as plt
import plotly.express as px

nombre_archivo = '../Datos/Iris.csv'

with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
X_raw = np.array(df.iloc[:, :4])
Y_raw = np.array(df['class'])

# Binarizador para convertir el nombre de la clase en one hot encoding
binarizer = preprocessing.LabelBinarizer()
Y_raw = binarizer.fit_transform(Y_raw)

scaler = preprocessing.StandardScaler()
#scaler = preprocessing.MinMaxScaler()
X_raw  = scaler.fit_transform( X_raw )

TEST_SIZE = 0.2
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_raw, Y_raw, test_size=TEST_SIZE)#, random_state=42)

print(X_raw)
print(Y_raw)


[[-9.00681170e-01  1.03205722e+00 -1.34127240e+00 -1.31297673e+00]
 [-1.14301691e+00 -1.24957601e-01 -1.34127240e+00 -1.31297673e+00]
 [-1.38535265e+00  3.37848329e-01 -1.39813811e+00 -1.31297673e+00]
 [-1.50652052e+00  1.06445364e-01 -1.28440670e+00 -1.31297673e+00]
 [-1.02184904e+00  1.26346019e+00 -1.34127240e+00 -1.31297673e+00]
 [-5.37177559e-01  1.95766909e+00 -1.17067529e+00 -1.05003079e+00]
 [-1.50652052e+00  8.00654259e-01 -1.34127240e+00 -1.18150376e+00]
 [-1.02184904e+00  8.00654259e-01 -1.28440670e+00 -1.31297673e+00]
 [-1.74885626e+00 -3.56360566e-01 -1.34127240e+00 -1.31297673e+00]
 [-1.14301691e+00  1.06445364e-01 -1.28440670e+00 -1.44444970e+00]
 [-5.37177559e-01  1.49486315e+00 -1.28440670e+00 -1.31297673e+00]
 [-1.26418478e+00  8.00654259e-01 -1.22754100e+00 -1.31297673e+00]
 [-1.26418478e+00 -1.24957601e-01 -1.34127240e+00 -1.44444970e+00]
 [-1.87002413e+00 -1.24957601e-01 -1.51186952e+00 -1.44444970e+00]
 [-5.25060772e-02  2.18907205e+00 -1.45500381e+00 -1.31297673e

In [3]:
# calcula las metricas precision, recall, f1-score y accuracy a partir de la matriz de confusion
# retorna tupla: ( precision, recall, f1_score, accuracy )
def calcular_metricas(conf_mat):
    precision = np.zeros(conf_mat.shape[0])
    for i in range(0, len(conf_mat)):
        precision[i] = conf_mat[i][i]/sum(conf_mat.T[i])

    recall = np.zeros(conf_mat.shape[0])
    for i in range(0, len(conf_mat)):
        recall[i] = conf_mat[i][i]/sum(conf_mat[i])

    f1_score = 2* (precision*recall) /(precision+recall)

    accuracy =  0
    for i in range(0, len(conf_mat)):
        accuracy+=conf_mat[i][i]
    accuracy/= conf_mat.sum()

    return ( precision, recall, f1_score, accuracy )

# el parámetro metricas es una tupla ( precision, recall, f1_score, accuracy )
def imprimir_metricas( metricas ):
    (precision, recall, f1_score, accuracy) = metricas
    print('\n clase   precision    recall    f1-score')
    for i in range(0, len(precision)):
        print('%5d %10.2f %10.2f %10.2f' % (i, precision[i], recall[i], f1_score[i]))
    print('\naccuracy: %6.2f\n' % accuracy)


In [8]:
ENTRADAS = X_train.shape[1]
OCULTAS = 2
SALIDAS  = Y_train.shape[1]
ACTIVACION = 'tanh'

print(X_train.shape)
print(X_test.shape)

EPOCAS = 500
TAM_LOTE = 32
ALFA = 0.02

modelo = Sequential()
modelo.add(Input(shape=(ENTRADAS,)))
modelo.add(Dense(OCULTAS, activation=ACTIVACION))
modelo.add(Dense(SALIDAS, activation='softmax'))

modelo.summary()

optimizador = optimizers.SGD(learning_rate=ALFA)
modelo.compile(optimizer=optimizador, loss='categorical_crossentropy', metrics = ['accuracy'])
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
history = modelo.fit(x=X_train, y=Y_train, batch_size=TAM_LOTE, epochs=EPOCAS, callbacks=[early_stop])

(120, 4)
(30, 4)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 2)                   │              10 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19 (76.00 B)

 Trainable params: 19 (76.00 B)

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9021 - loss: 0.3245 
Epoch 211/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8719 - loss: 0.3603  
Epoch 212/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8719 - loss: 0.3481  
Epoch 213/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8698 - loss: 0.3731  
Epoch 214/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8844 - loss: 0.3501  
Epoch 215/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8854 - loss: 0.3250  
Epoch 216/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8719 - loss: 0.3503  
Epoch 217/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8927 - loss: 0.3399  
Epoch 218/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8771 - loss: 0.3612 
Epoch 219/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8635 - loss: 0.3503 
Epoch 220/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8677 - loss: 0.3586 
Epoch 221/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.859

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9175 - loss: 0.2918 
Epoch 279/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8954 - loss: 0.2835  
Epoch 280/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.8798 - loss: 0.2977
Epoch 281/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9185 - loss: 0.2840 
Epoch 282/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9279 - loss: 0.2651  
Epoch 283/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9333 - loss: 0.2664 
Epoch 284/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9154 - loss: 0.2868  
Epoch 285/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.9187 - loss: 0.2891
Epoch 286/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9250 - loss: 0.2731  
Epoch 287/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9177 - loss: 0.2737  
Epoch 288/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9210 - loss: 0.2789 
Epoch 289/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.91

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9446 - loss: 0.2286 
Epoch 347/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9467 - loss: 0.2437  
Epoch 348/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9613 - loss: 0.2098 
Epoch 349/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9508 - loss: 0.2136 
Epoch 350/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9352 - loss: 0.2359 
Epoch 351/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9560 - loss: 0.2169 
Epoch 352/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9477 - loss: 0.2155 
Epoch 353/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9362 - loss: 0.2025 
Epoch 354/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9331 - loss: 0.2491 
Epoch 355/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9550 - loss: 0.2162 
Epoch 356/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9498 - loss: 0.2083 
Epoch 357/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.939

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9710 - loss: 0.1623 
Epoch 415/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9752 - loss: 0.1744 
Epoch 416/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9638 - loss: 0.1719 
Epoch 417/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9846 - loss: 0.1514 
Epoch 418/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9742 - loss: 0.1603 
Epoch 419/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9658 - loss: 0.1687 
Epoch 420/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9627 - loss: 0.1785 
Epoch 421/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9744 - loss: 0.1622 
Epoch 422/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9794 - loss: 0.1631 
Epoch 423/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9712 - loss: 0.1731 
Epoch 424/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9806 - loss: 0.1632 
Epoch 425/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.971

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9848 - loss: 0.1248  
Epoch 483/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9806 - loss: 0.1340 
Epoch 484/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9712 - loss: 0.1314  
Epoch 485/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9765 - loss: 0.1469 
Epoch 486/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9785 - loss: 0.1457 
Epoch 487/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9650 - loss: 0.1466  
Epoch 488/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9681 - loss: 0.1469 
Epoch 489/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9765 - loss: 0.1287 
Epoch 490/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9712 - loss: 0.1427 
Epoch 491/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9848 - loss: 0.1382  
Epoch 492/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9775 - loss: 0.1313 
Epoch 493/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.965

In [9]:
#  ########### Medición del entrenamiento ######################
Y_pred = modelo.predict(X_train)

# "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(Y_train)
Y_pred_it = binarizer.inverse_transform(Y_pred)

# calculo manual del accuracy
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )

report = metrics.classification_report(Y_it, Y_pred_it)
print("Reporte SkLearn:\n%s" % report)
cm = metrics.confusion_matrix(Y_it, Y_pred_it)
print("Confusion matrix:\n%s" % cm)

print("-------------------------")
print("mismos calculos de forma manual")
#calcula métricas de forma manual
mt = calcular_metricas(cm)
imprimir_metricas(mt)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Efectividad:  97.50%
Reporte SkLearn:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        42
Iris-versicolor       0.97      0.94      0.96        36
 Iris-virginica       0.95      0.98      0.96        42

       accuracy                           0.97       120
      macro avg       0.97      0.97      0.97       120
   weighted avg       0.98      0.97      0.97       120

Confusion matrix:
[[42  0  0]
 [ 0 34  2]
 [ 0  1 41]]
-------------------------
mismos calculos de forma manual

 clase   precision    recall    f1-score
    0       1.00       1.00       1.00
    1       0.97       0.94       0.96
    2       0.95       0.98       0.96

accuracy:   0.97

